In [82]:
import tensorflow as tf

def m_eye(n, k=0):
    """

    :param n: size of the matrix
    :param k: the shift value
    :return: a identity matrix where the ones aren shifted (to right) by k
     n=10 , k = 1,
tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    """
    return tf.cast(tf.concat((tf.concat(
                                (tf.zeros([n-k, k]),  tf.eye(n-k)), axis=1),
                         tf.zeros([k, n])), axis=0),tf.float32)
# def ctc_ent_loss(pred, token, pred_len,token_len, blank=0):
pred = tf.convert_to_tensor([[
	# BLANK     A	E		L		P
    [0.0,      0.8,	0, 		0, 		0.2],
    [0.0,      0.7, 0, 		0.3, 	0],
    [0.9,      0, 	0, 		0, 		0.1],
    [0.1,      0, 	0, 		0, 		0.9],
    [0.9,      0, 	0, 		0.1, 	0],
    [0.0,      0.1, 0.1, 	0, 		0.8],
    [0.1,      0, 	0, 		.8, 	0.1],
    [0.0,      0,	.8, 	.1, 	0.1]
],[
	# BLANK     A	E		L		P
    [0.0,      0.8,	0, 		0, 		0.2],
    [0.0,      0.7, 0, 		0.3, 	0],
    [0.9,      0, 	0, 		0, 		0.1],
    [0.1,      0, 	0, 		0, 		0.9],
    [0.9,      0, 	0, 		0.1, 	0],
    [0.0,      0.1, 0.1, 	0, 		0.8],
    [0.1,      0, 	0, 		.8, 	0.1],
    [0.0,      0,	.8, 	.1, 	0.1]
]],tf.float64)

In [83]:
# pred = pred[:,:]
pred.shape

TensorShape([2, 8, 5])

In [84]:
token = tf.convert_to_tensor([[1,4,4,3,2],[1,4,4,3,2]])

In [85]:
if True:
    '''
    :param pred: ( batch,Time, voca_size+1)
    :param pred_len: (batch,1)
    :param token: (batch, U), U : length maximal of sequence in batch, padded with ?, length known with token_len
    :param token_len: (batch,1)
    # blank seem the index of the blank, but use T.zeros for token_with_blank

    :out alpha: (Time, batch, 2U+1) ∑p(π|x)
    :out beta: (Time, batch, 2U+1)  ∑p(π|x)logp(π|x)
    :out H: -beta/alpha+log(alpha)
    '''
    # print(pred)
    pred = tf.transpose(pred,[1,0,2]) # ( Time,batch, voca_size+1)
    Time, batch = tf.shape(pred)[0], tf.shape(pred)[1]
    U = tf.shape(token)[1]
    eps = 1E-8

In [86]:
    # token_with_blank
    # "view" is a kind of reshape with sharing memory, but here the original concat is not kept, so same as reshape i guess
    token_with_blank = tf.concat((tf.zeros([batch, U, 1],dtype=tf.int32), tf.cast(token[:, :, tf.newaxis],tf.int32)),axis=2)  # (batch, U,2)
    token_with_blank = tf.reshape(token_with_blank,[batch,-1])# (batch, 2*U)
    # add a blank at the end of elems
    token_with_blank = tf.concat((token_with_blank, tf.zeros([batch, 1],dtype=tf.int32)), axis=1)  # (batch, 2U+1)
    # token_with_blank: [blank, index_e1,blank, index_e2, .... eU, blank]

    length = tf.shape(token_with_blank)[1]  # 2U+1

In [87]:
print(token_with_blank)

tf.Tensor(
[[0 1 0 4 0 4 0 3 0 2 0]
 [0 1 0 4 0 4 0 3 0 2 0]], shape=(2, 11), dtype=int32)


In [88]:
if True:
    # take the scores of elements of the token, at each step for each batch
    """
    In[24]:    a
    Out[24]:
    <shape=(1 (Time), 2 (batch), 5),
    array([[[0.9688189 , 0.01720107, 0.5965241 , 0.76499116, 0.6780566 ],
            [0.08494377, 0.34031677, 0.92728245, 0.83308065, 0.17265546]]],
    In[25]:
    tf.gather(a,tf.convert_to_tensor([
                                        [0,2,4], #batch1
                                        [0,1,4]] #batch2
                                    )[tf.newaxis,:],axis=2,batch_dims=2)
    Out[25]:
    <shape=(1, 2, 3),
    array([[[0.9688189 , 0.5965241 , 0.6780566 ],
            [0.08494377, 0.34031677, 0.17265546]]]
    """
    pred = tf.gather(pred,tf.repeat(token_with_blank[tf.newaxis,:,:],Time,axis=0),axis=2,batch_dims=2)  # (T, batch, 2U+1)]
    # print(pred)

In [89]:
    print(pred)

tf.Tensor(
[[[0.  0.8 0.  0.2 0.  0.2 0.  0.  0.  0.  0. ]
  [0.  0.8 0.  0.2 0.  0.2 0.  0.  0.  0.  0. ]]

 [[0.  0.7 0.  0.  0.  0.  0.  0.3 0.  0.  0. ]
  [0.  0.7 0.  0.  0.  0.  0.  0.3 0.  0.  0. ]]

 [[0.9 0.  0.9 0.1 0.9 0.1 0.9 0.  0.9 0.  0.9]
  [0.9 0.  0.9 0.1 0.9 0.1 0.9 0.  0.9 0.  0.9]]

 [[0.1 0.  0.1 0.9 0.1 0.9 0.1 0.  0.1 0.  0.1]
  [0.1 0.  0.1 0.9 0.1 0.9 0.1 0.  0.1 0.  0.1]]

 [[0.9 0.  0.9 0.  0.9 0.  0.9 0.1 0.9 0.  0.9]
  [0.9 0.  0.9 0.  0.9 0.  0.9 0.1 0.9 0.  0.9]]

 [[0.  0.1 0.  0.8 0.  0.8 0.  0.  0.  0.1 0. ]
  [0.  0.1 0.  0.8 0.  0.8 0.  0.  0.  0.1 0. ]]

 [[0.1 0.  0.1 0.1 0.1 0.1 0.1 0.8 0.1 0.  0.1]
  [0.1 0.  0.1 0.1 0.1 0.1 0.1 0.8 0.1 0.  0.1]]

 [[0.  0.  0.  0.1 0.  0.1 0.  0.1 0.  0.8 0. ]
  [0.  0.  0.  0.1 0.  0.1 0.  0.1 0.  0.8 0. ]]], shape=(8, 2, 11), dtype=float64)


In [90]:
    # ## recurrence relation
    # take elements with (not) consecutive same token -> 'aa' -> need to pass by the blank
    # True/1 = different, False/0 = same
    print(token_with_blank)
    print("   ",token_with_blank[:, :-2])
    print("   ",token_with_blank[:, 2:])
    consecutiveDifferent = tf.cast(tf.not_equal(token_with_blank[:, :-2], token_with_blank[:, 2:]),tf.float32)

    # pad with two blanks on the left
    consecutiveDifferent = tf.concat(( tf.zeros((batch, 2),dtype=tf.float32), consecutiveDifferent), axis=1)
    print(tf.cast(consecutiveDifferent,tf.int32))

tf.Tensor(
[[0 1 0 4 0 4 0 3 0 2 0]
 [0 1 0 4 0 4 0 3 0 2 0]], shape=(2, 11), dtype=int32)
    tf.Tensor(
[[0 1 0 4 0 4 0 3 0]
 [0 1 0 4 0 4 0 3 0]], shape=(2, 9), dtype=int32)
    tf.Tensor(
[[0 4 0 4 0 3 0 2 0]
 [0 4 0 4 0 3 0 2 0]], shape=(2, 9), dtype=int32)
tf.Tensor(
[[0 0 0 1 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 1 0 1 0]], shape=(2, 11), dtype=int32)


In [91]:
    print(token_with_blank)
    # elements not blank in the GT (one on two)
    notBlanksToken = tf.cast(tf.not_equal(token_with_blank, 0),tf.float32)
    print(notBlanksToken)

tf.Tensor(
[[0 1 0 4 0 4 0 3 0 2 0]
 [0 1 0 4 0 4 0 3 0 2 0]], shape=(2, 11), dtype=int32)
tf.Tensor(
[[0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]], shape=(2, 11), dtype=float32)


In [92]:
    # kind of mask : True = Not consecutive identic elements, False = consecutive identic elements: can skip the blank,
    sec_diag = consecutiveDifferent * notBlanksToken  # (batch, 2U+1)
    print(sec_diag)

tf.Tensor(
[[0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]], shape=(2, 11), dtype=float32)


In [97]:
    # m_eye : identity matrix for k=0, ones of are shifted by k

    # recurrence_relation : (batch, 2U+1, 2U+1)
    recurrence_relation = \
        tf.repeat((tf.eye(length) + m_eye(length, k=1))[tf.newaxis], repeats = batch,axis=0) + \
        tf.repeat(  m_eye(length, k=2)[tf.newaxis],                  repeats = batch,axis=0) * sec_diag[:, tf.newaxis, :]
    # tf.print("recurrence_relation\n",recurrence_relation,summarize=-1)
    print(tf.squeeze(recurrence_relation))
    ### end recurrence relation
    #%%
    # print((tf.cast(tf.logical_not(tf.eye(length,dtype=tf.bool)),tf.float32))+((tf.transpose(tf.cast(tf.logical_not(tf.cast(notBlanksToken,tf.bool)),tf.float32)*(0.8)))))
    eyed = tf.repeat(tf.eye(length)[tf.newaxis], repeats = batch,axis=0)
    tf.print(tf.shape(recurrence_relation),tf.shape(eyed),tf.shape(notBlanksToken))
    # print(recurrence_relation,tf.eye(length, dtype=tf.float32),notBlanksToken)
    weightInverse = 0.2
    print(recurrence_relation*(1-(eyed)*notBlanksToken[:,:,tf.newaxis]*weightInverse))

tf.Tensor(
[[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

 [[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]], shape=(2, 11, 11), dtype=float32)
[2 11 11] [2 11 11] [2 11]
tf.Tensor(
[[[1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
  [0.  0.8 1.  1.  0.  0.  0.  0.  0.  0.  0. ]
  

In [78]:
    eps_nan = -1e8

    eps_nan * (tf.ones_like(recurrence_relation) - recurrence_relation)

<tf.Tensor: shape=(1, 11, 11), dtype=float32, numpy=
array([[[-0.e+00, -0.e+00, -1.e+08, -1.e+08, -1.e+08, -1.e+08, -1.e+08,
         -1.e+08, -1.e+08, -1.e+08, -1.e+08],
        [-1.e+08, -0.e+00, -0.e+00, -0.e+00, -1.e+08, -1.e+08, -1.e+08,
         -1.e+08, -1.e+08, -1.e+08, -1.e+08],
        [-1.e+08, -1.e+08, -0.e+00, -0.e+00, -1.e+08, -1.e+08, -1.e+08,
         -1.e+08, -1.e+08, -1.e+08, -1.e+08],
        [-1.e+08, -1.e+08, -1.e+08, -0.e+00, -0.e+00, -1.e+08, -1.e+08,
         -1.e+08, -1.e+08, -1.e+08, -1.e+08],
        [-1.e+08, -1.e+08, -1.e+08, -1.e+08, -0.e+00, -0.e+00, -1.e+08,
         -1.e+08, -1.e+08, -1.e+08, -1.e+08],
        [-1.e+08, -1.e+08, -1.e+08, -1.e+08, -1.e+08, -0.e+00, -0.e+00,
         -0.e+00, -1.e+08, -1.e+08, -1.e+08],
        [-1.e+08, -1.e+08, -1.e+08, -1.e+08, -1.e+08, -1.e+08, -0.e+00,
         -0.e+00, -1.e+08, -1.e+08, -1.e+08],
        [-1.e+08, -1.e+08, -1.e+08, -1.e+08, -1.e+08, -1.e+08, -1.e+08,
         -0.e+00, -0.e+00, -0.e+00, -1.e+08],
   

In [41]:
    # alpha
    # initialisation, two first char of seq (blank and first elem) initalised with the direct score of prediction
    """
    T.cat((pred[0,:,:2],T.zeros(batch,5-1)),dim=1)
        tensor([
        [0.9503, 0.3439, 0.0000, 0.0000, 0.0000, 0.0000], # 1er elem of batch
        [0.7988, 0.6641, 0.0000, 0.0000, 0.0000, 0.0000]]) # 2nd elem of batch

    --> For t = 0, take blank and first elem of token
    """
    alpha_t = tf.concat((pred[0, :, :2], tf.zeros([batch, 2 * U + 1 - 2],dtype=tf.double)), axis=1)  # (batch, 2U+1)
    # same but score*log(score) : this is the regularisation
    beta_t = tf.concat((pred[0, :, :2] * tf.math.log(pred[0, :, :2]), tf.zeros([batch, 2 * U - 1],dtype=tf.double)),
                   axis=1)  # (batch, 2U+1)
    print(alpha_t)

tf.Tensor([[0.  0.8 0.  0.  0.  0.  0.  0.  0.  0.  0. ]], shape=(1, 11), dtype=float64)


In [42]:
    print(pred)

tf.Tensor(
[[[0.  0.8 0.  0.2 0.  0.2 0.  0.  0.  0.  0. ]]

 [[0.  0.7 0.  0.  0.  0.  0.  0.3 0.  0.  0. ]]

 [[0.9 0.  0.9 0.1 0.9 0.1 0.9 0.  0.9 0.  0.9]]

 [[0.1 0.  0.1 0.9 0.1 0.9 0.1 0.  0.1 0.  0.1]]

 [[0.9 0.  0.9 0.  0.9 0.  0.9 0.1 0.9 0.  0.9]]

 [[0.  0.1 0.  0.8 0.  0.8 0.  0.  0.  0.1 0. ]]

 [[0.1 0.  0.1 0.1 0.1 0.1 0.1 0.8 0.1 0.  0.1]]

 [[0.  0.  0.  0.1 0.  0.1 0.  0.1 0.  0.8 0. ]]], shape=(8, 1, 11), dtype=float64)


In [ ]:
    alphas = tf.zeros([Time,batch,length])
    betas = tf.zeros([Time,batch,length])
    # will store all the alphas for all times, first do it for t=0
    """
    alpha_t =  tf.convert_to_tensor([[1,2,0,0],[5,6,0,0]],dtype=tf.float32)
    tf.tensor_scatter_nd_update(tf.zeros(shape=(3, 2, 4)), [[0]],alpha_t[tf.newaxis])
    Out:
    <tf.Tensor: shape=(3, 2, 4), dtype=float32, numpy=
    array([[[1., 2., 0., 0.],
            [5., 6., 0., 0.]],
           [[0., 0., 0., 0.],
            [0., 0., 0., 0.]],
           [[0., 0., 0., 0.],
            [0., 0., 0., 0.]]], dtype=float32)>
    """
    alphas = tf.tensor_scatter_nd_update(alphas, [[0]], alpha_t[tf.newaxis])
    betas = tf.tensor_scatter_nd_update(betas, [[0]], beta_t[tf.newaxis])
    # tf.print("pred",pred,summarize=-1)

In [ ]:
    # dynamic programming
    # (T, batch, 2U+1)
    def do(t):
        global alpha_t
        global beta_t
        alpha_t = tf.matmul(alpha_t[:, tf.newaxis], recurrence_relation)[:, 0] * pred[t, :, :]
        beta_t = tf.matmul(beta_t[:, tf.newaxis], recurrence_relation)[:, 0] * pred[t, :, :] + tf.math.log(
            pred[t] + eps) * alpha_t
        global alphas
        global betas
        alphas = tf.tensor_scatter_nd_update(alphas, [[t]], alpha_t[tf.newaxis])
        betas = tf.tensor_scatter_nd_update(betas, [[t]], beta_t[tf.newaxis])
        return tf.add(t, 1)

    tf.map_fn(do,tf.range(1, Time,dtype=tf.int32))
    #
    # for t in tf.range(1, Time,dtype=tf.int32):
    #     alpha_t = tf.matmul(alpha_t[:, tf.newaxis], recurrence_relation)[:, 0] * pred[t,:,:]
    #     beta_t = tf.matmul(beta_t[:, tf.newaxis], recurrence_relation)[:, 0] * pred[t,:,:] + tf.math.log(pred[t]+eps) * alpha_t
    #
    #     alphas = tf.tensor_scatter_nd_update(alphas, [[t]], alpha_t[tf.newaxis])
    #     betas = tf.tensor_scatter_nd_update(betas, [[t]], beta_t[tf.newaxis])

    # tf.print("alphas", Time, alphas)
    # tf.print("beta_t", Time-1, beta_t)

    def collect_label(probability):
        probability = tf.transpose(probability,[1,0,2])
        #pred len : [batch,1]
        probability = tf.gather(probability, pred_len-1, axis=1, batch_dims=1)[:,0,:]
        #token_len : [batch,1]
        labels_2=tf.gather(probability, token_len-2, axis=1, batch_dims=1)[:,0]
        # labels_2 = probability[Time - 1, :, length-2]#last true elem
        labels_1=tf.gather(probability, token_len-1, axis=1, batch_dims=1)[:,0]
        # labels_1 = probability[Time - 1, :, length-1] #blank
        labels_prob = labels_2 + labels_1  # sum of the two possible ends : last elem of token or blank
        return labels_prob

    alpha = collect_label(alphas)
    beta = collect_label(betas)
    # tf.print("alpha final",  alpha,summarize=-1)

    tf.print("alpha",alpha)
    # tf.print("beta",beta)

    H = -beta / (alpha+eps) + tf.math.log(alpha + eps)
    costs = -tf.math.log(alpha + eps)
    valFinal = tf.reduce_mean(costs )# - 0.2*H)
    # tf.print("H",H,summarize=-1)
    tf.print("valFinal",valFinal,summarize=-1)
    # tf.print("valFinal",valFinal)

In [ ]:
def log_batch_dot(alpha_t, rec):
    '''
    alpha_t: (batch, 2U+1)
    rec: (batch, 2U+1, 2U+1)
    '''
    # tf.print("log_batch_dot")

    eps_nan = -1e8
    # a+b
    _sum = alpha_t[:, :, None] + rec
    _max_sum = tf.reduce_max(_sum, axis=1)# 2U+1
    nz_mask1 = tf.greater(_max_sum, eps_nan) # max > eps_nan
    nz_mask2 = tf.greater(_sum, eps_nan)     # item > eps_nan
    # tf.print("nz_mask1",nz_mask1,nz_mask1.shape,summarize=-1)
    # tf.print("nz_mask2",nz_mask2,nz_mask2.shape,summarize=-1)

    # a+b-max
    _sum = _sum - _max_sum[:, None]

    # exp
    _exp = tf.zeros_like(_sum,dtype=tf.float32)#.type(floatX)
    # _exp[nz_mask2] = tf.exp(_sum[nz_mask2])
    _exp = tf.tensor_scatter_nd_update(_exp,tf.where(nz_mask2),tf.exp(_sum[nz_mask2]))
    # tf.print("_exp",_exp,tf.shape(_exp),"exp__",summarize=-1)

    # sum exp
    _sum_exp = tf.reduce_sum(_exp, axis=1)

    out = tf.ones_like(_max_sum,dtype=tf.float32) * eps_nan
    # out[nz_mask1] = tf.math.log(_sum_exp[nz_mask1]) + _max_sum[nz_mask1]
    out = tf.tensor_scatter_nd_update(out,tf.where(nz_mask1),tf.math.log(_sum_exp[nz_mask1]) + _max_sum[nz_mask1])

    return out

def log_sum_exp_axis(a, uniform_mask=None, dim=0):
    assert dim == 0
    """
    a : [2,batch]
    """
    eps_nan = -1e8
    eps = 1e-26
    _max = tf.reduce_max(a, axis=dim)

    if not uniform_mask is None:
        nz_mask2 = tf.greater(a, eps_nan) * uniform_mask
        nz_mask1 = tf.greater(_max, eps_nan) * tf.greater_equal(tf.reduce_max(uniform_mask, axis=dim), 1)
    else:
        nz_mask2 = tf.greater(a, eps_nan) #
        nz_mask1 = tf.greater(_max, eps_nan)
    # tf.print("nz_mask2.shape",tf.shape(nz_mask2))
    # tf.print("nz_mask1.shape",tf.shape(nz_mask1))
    # a-max
    a = a - _max[None]

    # exp
    _exp_a = tf.zeros_like(a,dtype=tf.float32)
    # _exp_a[nz_mask2] = tf.exp(a[nz_mask2])
    _exp_a = tf.tensor_scatter_nd_update(_exp_a,tf.where(nz_mask2),tf.exp(a[nz_mask2]))

    # sum exp
    _sum_exp_a = tf.reduce_sum(_exp_a, axis=dim)

    out = tf.ones_like(_max,dtype=tf.float32) * eps_nan
    # out[nz_mask1] =
    summ = _sum_exp_a[nz_mask1]
    maxx = _max[nz_mask1]
    # tf.print(nz_mask1)
    val = tf.math.log( summ+ eps) + maxx
    out = tf.tensor_scatter_nd_update(out,tf.where(nz_mask1),val)
    return out

def log_sum_exp(*arrs):
    """
    probability :
    :param arrs:[batch], [batch]
    :return:
    """
#    return T.max(a.clone(), b.clone()) + T.log1p(T.exp(-T.abs(a.clone()-b.clone())))
    c = tf.concat(list(map(lambda x:x[None], arrs)), axis=0) #[2,batch]
    # tf.print("c.shape",tf.shape(c))
    return log_sum_exp_axis(c, dim=0)

@tf.function
def ctc_ent_loss_log(pred, token, pred_len,token_len, blank=0):
    '''
    :param pred: ( batch,Time, voca_size+1)
    :param pred_len: (batch)
    :param token: (batch, U), U : length maximal of sequence in batch, padded with ?, length known with token_len
    :param token_len: (batch)
    # blank seem the index of the blank, but use T.zeros for token_with_blank

    :out alpha: (Time, batch, 2U+1) ∑p(π|x)
    :out beta: (Time, batch, 2U+1)  ∑p(π|x)logp(π|x)
    :out H: -beta/alpha+log(alpha)
    '''
    pred = tf.transpose(pred,[1,0,2]) # ( Time,batch, voca_size+1)
    pred = tf.math.log_softmax(pred)
    Time, batch = tf.shape(pred)[0], tf.shape(pred)[1]
    U = tf.shape(token)[1]
    eps = 1E-8
    eps_nan = -1e8

    # token_with_blank
    # "view" is a kind of reshape with sharing memory, but here the original concat is not kept, so same as reshape i guess
    token_with_blank = tf.concat((tf.zeros([batch, U, 1],dtype=tf.int32), tf.cast(token[:, :, tf.newaxis],tf.int32)),axis=2)  # (batch, U,2)
    token_with_blank = tf.reshape(token_with_blank,[batch,-1])# (batch, 2*U)
    # add a blank at the end of elems
    token_with_blank = tf.concat((token_with_blank, tf.zeros([batch, 1],dtype=tf.int32)), axis=1)  # (batch, 2U+1)
    # token_with_blank: [blank, index_e1,blank, index_e2, .... eU, blank]

    length = tf.shape(token_with_blank)[1]  # 2U+1

    # take the scores of elements of the token, at each step for each batch
    """
    In[24]:    a
    Out[24]:
    <shape=(1 (Time), 2 (batch), 5),
    array([[[0.9688189 , 0.01720107, 0.5965241 , 0.76499116, 0.6780566 ],
            [0.08494377, 0.34031677, 0.92728245, 0.83308065, 0.17265546]]],
    In[25]:
    tf.gather(a,tf.convert_to_tensor([
                                        [0,2,4], #batch1
                                        [0,1,4]] #batch2
                                    )[tf.newaxis,:],axis=2,batch_dims=2)
    Out[25]:
    <shape=(1, 2, 3),
    array([[[0.9688189 , 0.5965241 , 0.6780566 ],
            [0.08494377, 0.34031677, 0.17265546]]]
    """
    pred = tf.gather(pred,tf.repeat(token_with_blank[tf.newaxis,:,:],Time,axis=0),axis=2,batch_dims=2)  # (T, batch, 2U+1)]
    # print(pred)

    # ## recurrence relation
    # take elements with (not) consecutive same token -> 'aa' -> need to pass by the blank
    # True/1 = different, False/0 = same
    consecutiveDifferent = tf.cast(tf.not_equal(token_with_blank[:, :-2], token_with_blank[:, 2:]),tf.float32)

    # pad with two blanks on the left
    consecutiveDifferent = tf.concat(( tf.zeros((batch, 2),dtype=tf.float32), consecutiveDifferent), axis=1)

    # elements not blank in the GT (one on two)
    notBlanksToken = tf.cast(tf.not_equal(token_with_blank, blank),tf.float32)
    # kind of mask : True = Not consecutive identic elements, False = consecutive identic elements: can skip the blank,
    sec_diag = consecutiveDifferent * notBlanksToken  # (batch, 2U+1)

    # m_eye : identity matrix for k=0, ones of are shifted by k

    # recurrence_relation : (batch, 2U+1, 2U+1)
    recurrence_relation = \
        tf.repeat((tf.eye(length) + m_eye(length, k=1))[tf.newaxis], repeats = batch,axis=0) + \
        tf.repeat(  m_eye(length, k=2)[tf.newaxis],                  repeats = batch,axis=0) * sec_diag[:, tf.newaxis, :]
    # tf.print("recurrence_relation\n",recurrence_relation,summarize=-1)
    recurrence_relation = eps_nan * (tf.ones_like(recurrence_relation) - recurrence_relation)

    ### end recurrence relation

    # alpha
    # initialisation, two first char of seq (blank and first elem) initalised with the direct score of prediction
    """
    T.cat((pred[0,:,:2],T.zeros(batch,5-1)),dim=1)
        tensor([
        [0.9503, 0.3439, 0.0000, 0.0000, 0.0000, 0.0000], # 1er elem of batch
        [0.7988, 0.6641, 0.0000, 0.0000, 0.0000, 0.0000]]) # 2nd elem of batch

    --> For t = 0, take blank and first elem of token
    """
    alpha_t = tf.concat((pred[0, :, :2], tf.ones([batch, 2 * U + 1 - 2],dtype=tf.float32)*eps_nan), axis=1)  # (batch, 2U+1)
    # same but score*log(score) : this is the regularisation
    beta_t = tf.concat((pred[0, :, :2] * tf.math.log(-pred[0, :, :2]+eps), tf.ones([batch, 2 * U - 1],dtype=tf.float32)*eps_nan),
                   axis=1)  # (batch, 2U+1)
    # tf.print("alpha t 0",alpha_t,tf.shape(alpha_t))

    alphas = tf.zeros([Time,batch,length])
    betas = tf.zeros([Time,batch,length])
    # will store all the alphas for all times, first do it for t=0
    """
    alpha_t =  tf.convert_to_tensor([[1,2,0,0],[5,6,0,0]],dtype=tf.float32)
    tf.tensor_scatter_nd_update(tf.zeros(shape=(3, 2, 4)), [[0]],alpha_t[tf.newaxis])
    Out:
    <tf.Tensor: shape=(3, 2, 4), dtype=float32, numpy=
    array([[[1., 2., 0., 0.],
            [5., 6., 0., 0.]],
           [[0., 0., 0., 0.],
            [0., 0., 0., 0.]],
           [[0., 0., 0., 0.],
            [0., 0., 0., 0.]]], dtype=float32)>
    """
    alphas = tf.tensor_scatter_nd_update(alphas, [[0]], alpha_t[tf.newaxis])
    # betas = tf.tensor_scatter_nd_update(betas, [[0]], beta_t[tf.newaxis])
    # tf.print("pred",pred,summarize=-1)
    # print(tf.executing_eagerly())

    def do(t, alpha_t, beta_t, alphas, betas):
        alpha_t = log_batch_dot(alpha_t, recurrence_relation) + pred[t, :, :]
        beta_t = log_sum_exp(log_batch_dot(beta_t, recurrence_relation) + pred[t, :, :],
                             tf.math.log(-pred[t] + eps) + alpha_t)
        alphas = tf.tensor_scatter_nd_update(alphas, [[t]], alpha_t[tf.newaxis])
        betas = tf.tensor_scatter_nd_update(betas, [[t]], beta_t[tf.newaxis])
        return t + 1, alpha_t, beta_t, alphas, betas

    i = tf.constant(0)
    i, alpha_t, beta_t, alphas, betas, = tf.while_loop(lambda i, at, bt, a, b: i < Time, do,
                                                       [i, alpha_t, beta_t, alphas, betas, ])

    # dynamic programming
    # # (T, batch, 2U+1)
    # for t in tf.range(1, Time,dtype=tf.int32):
    #     alpha_t = log_batch_dot(alpha_t, recurrence_relation)+ pred[t,:,:]
    #     beta_t = log_sum_exp(log_batch_dot(beta_t, recurrence_relation) + pred[t,:,:],
    #                          tf.math.log(-pred[t]+eps) + alpha_t)
    #
    #     alphas = tf.tensor_scatter_nd_update(alphas, [[t]], alpha_t[tf.newaxis])
    #     betas = tf.tensor_scatter_nd_update(betas, [[t]], beta_t[tf.newaxis])
    #     # tf.print("alphas t", t,alphas, tf.shape(alphas),summarize=-1)
    #
    # tf.print("alphas", alphas,tf.shape(alphas))

    # tf.print("alphas", Time, alphas)
    # tf.print("beta_t", Time-1, beta_t)

    def collect_label(probability):
        """

        :param probability: [time,batch, 2U+1]
        :return:
        """
        probability = tf.transpose(probability, [1, 0, 2])
        # pred len : [batch,1]
        # print(probability)
        # print(pred_len)
        probability = tf.gather(probability, pred_len - 1, axis=1, batch_dims=1)[:, 0, :]
        # probability = probability[Time - 1, :, :]  # last true elem
        # token_len : [batch,1]
        realTokenlen = (token_len * 2 + 1)
        labels_2 = tf.gather(probability,  realTokenlen- 2, axis=1, batch_dims=1)[:, 0]
        # labels_2 = probability[:, length-2]#last true elem
        labels_1 = tf.gather(probability, realTokenlen - 1, axis=1, batch_dims=1)[:, 0]
        # labels_1 = probability[:, length-1] #blank

        labels_2 = tf.repeat(labels_2[tf.newaxis,:],batch,axis=0)#last true elem
        labels_1 = tf.repeat(labels_1[tf.newaxis,:],batch,axis=0)#last true elem
        # labels_1 = tf.repeat(probability[token_len - 1,tf.newaxis, :, length-1],batch,axis=0)#last true elem
        # tf.print("labels_2", tf.shape(labels_2))
        # tf.print("labels_1", tf.shape(labels_1))

        labels_prob = log_sum_exp(labels_2,labels_1)  # sum of the two possible ends : last elem of token or blank
        return labels_prob

    alpha = collect_label(alphas)
    # tf.print("alpha",alpha,tf.shape(alpha))
    beta = collect_label(betas)
    # tf.print("alpha final",  alpha,summarize=-1)

    # tf.print("alpha",alpha)
    # tf.print("beta",beta)

    H = tf.exp(beta-alpha)+alpha
    costs = -alpha
    valFinal = 0.8*tf.reduce_sum(costs) - 0.2*tf.reduce_sum(H)
    # tf.print("H",H,summarize=-1)
    # tf.print("valFinal",valFinal,summarize=-1)
    # tf.print("valFinal",valFinal)
    return valFinal
